# Extract & Load IPC data
*Indice des Prix à la Consommation au Maroc*

**Source** : [Annuaire Statistique du Maroc](https://www.hcp.ma/downloads/?tag=Annuaires+statistiques+du+Maroc)

**Années** : `2020-2022`

## Setup

### Import statements

In [1]:
import os
import pathlib
import re
import string

import duckdb
import openpyxl
import pandas as pd
import plotly.express as px
from tqdm import tqdm

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

### All paths used in the project

In [108]:
data_path = pathlib.Path(pathlib.Path.cwd(), "data")
source_data_path = pathlib.Path(data_path, "source")
raw_data_path = pathlib.Path(data_path, "raw")
joined_data_path = pathlib.Path(data_path, "joined")
clean_data_path = pathlib.Path(data_path, "clean")
duckdb_db_path = pathlib.Path(data_path, "duckdb/duckdb.duckdb")

### Parameters

In [10]:
files_list = {
    "IPC_2020":{
        "file_name":"indice_des_prix_2020.xlsx",
        "nb_cols":3
    },
    "IPC_2021":{
        "file_name":"indice_des_prix_2021.xlsx",
        "nb_cols":4
    },
    "IPC_2022":{
        "file_name":"indice_des_prix_2022.xlsx",
        "nb_cols":4
    }
}

produit_column_name = "produit"

national_sheet_name = "Nat "

skiprows_end = 8
skipfooter = 6


### Functions toolkit

In [9]:
def get_excel_file(folder_path: pathlib.Path, excel_file_name: str) -> pd.ExcelFile:
    excel_file_path = pathlib.Path(folder_path, excel_file_name)
    return pd.ExcelFile(excel_file_path)


def generate_columns_with_years(year_to_extract: int, nb_cols:int ) -> list:
    col_produit = [produit_column_name]
    col_years = [f"year_{x}" for x in range(year_to_extract-1, year_to_extract-nb_cols, -1)]
    return col_produit + col_years


def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    # clean produits
    col_years=list(df.columns)[1:]
    ls_produits = []
    previous = ""
    for i, row in enumerate(df.itertuples()):
        actual = row.produit
        elem = getattr(row, f"{col_years[0]}")
        if pd.isna(elem):
            previous = actual.strip()
        elif previous:
            ls_produits.append(previous + " " + actual.strip())
            previous = ""
        else:
            ls_produits.append(actual.strip())
            previous = ""

    df_produits = pd.DataFrame(ls_produits, columns=[produit_column_name])
    
    # clean values
    df_values = (
        df[df.columns.difference([produit_column_name])]
        .dropna(
            axis=0,
            how='any',
        )
        .reset_index(drop=True)
    )
    
    return pd.concat([df_produits, df_values], axis=1)


def extract_year_from_filename(file_name:str) -> int:
    return int(re.search(r"\d{4}", file_name).group())


def get_column_range(col_number:int) -> str:
    return f"A:{string.ascii_uppercase[col_number-1]}"

## Extract & Load data

In [110]:
file_iterator = tqdm(files_list.keys())
for year_label in file_iterator:
    year_dict = files_list[year_label]
    file_name = year_dict["file_name"]
    nb_cols = year_dict["nb_cols"]
    file_iterator.set_description(f"Accessing {file_name}..")
    excel_file = get_excel_file(
        folder_path=source_data_path,
        excel_file_name=file_name
    )

    year_to_extract = extract_year_from_filename(file_name)

    columns_list = generate_columns_with_years(year_to_extract, nb_cols)

    df = pd.read_excel(
        excel_file,
        national_sheet_name, 
        skiprows=range(1, skiprows_end),
        usecols=get_column_range(nb_cols),
        names=columns_list,
        skipfooter=skipfooter
    )
    file_iterator.set_description(f"Cleaning {file_name}..")
    clean_df = clean_dataframe(df)

    destination_file_name = f"indice_des_prix_{year_to_extract}.parquet"
    file_iterator.set_description(f"Writing data to {destination_file_name}..")
    destination_file_path = pathlib.Path(raw_data_path,destination_file_name)

    clean_df.to_parquet(destination_file_path)


Writing data to indice_des_prix_2022.parquet..: 100%|██████████| 3/3 [00:01<00:00,  1.65it/s]


## Join data using duckdb

In [7]:
%%sql duckdb:///{duckdb_db_path.relative_to(os.getcwd()).as_posix()}
INSTALL parquet;
LOAD parquet;

""


In [11]:
dc_raw = {extract_year_from_filename(file_name):f"{raw_data_path}/{file_name}" \
    for file_name in os.listdir(raw_data_path)
}
dc_raw

{2020: '/jupyter_lab/notebooks/statistiques_maroc/data/raw/indice_des_prix_2020.parquet',
 2021: '/jupyter_lab/notebooks/statistiques_maroc/data/raw/indice_des_prix_2021.parquet',
 2022: '/jupyter_lab/notebooks/statistiques_maroc/data/raw/indice_des_prix_2022.parquet'}

### Merge Data From IPC 2020 to latest

In [12]:
%%sql joined_df <<

WITH IPC_2020 AS (
    SELECT * FROM "{dc_raw[2020]}"
),

IPC_2021 AS (
    SELECT * FROM "{dc_raw[2021]}"
),

IPC_2022 AS (
    SELECT * FROM "{dc_raw[2022]}"
)

SELECT 
    IPC_2020.produit,
    100::FLOAT AS year_2017,
    IPC_2020.year_2018::FLOAT AS year_2018,
    IPC_2021.year_2019::FLOAT AS year_2019,
    IPC_2021.year_2020::FLOAT AS year_2020,
    IPC_2022.year_2021::FLOAT AS year_2021
FROM IPC_2020
JOIN IPC_2021 ON IPC_2021.produit = IPC_2020.produit
JOIN IPC_2022 ON IPC_2022.produit = IPC_2020.produit

In [13]:
joined_destination_path = pathlib.Path(joined_data_path, 'IPC.parquet')
joined_df.to_parquet(joined_destination_path)

## Pivot data using pandas

In [15]:
pivoted_df = pd.melt(joined_df, 
   id_vars = 'produit', 
   value_vars = [f"year_{x}" for x in range(2017, 2022)], 
   var_name = 'str_year', 
   value_name = 'value'
)



In [16]:
pivoted_df["year"] = pivoted_df["str_year"].str.extract(r"year_(?P<year>\d{4})", expand=False)
pivoted_df = pivoted_df[pivoted_df.columns.difference(["str_year"])]

## Process Code-Label Mapping
Le panier de l’IPC contient 546 articles et 1391 variétés de produits représentant la majorité des articles consommés par la population urbaine.  
Ces articles sont classés en 
- 12 divisions
- 43 groupes
- 92 sous groupes *(comptabilisés manuellement)*

L'IPC l'annuaire statistique :
- 
ne contient pas de groupes
- ne contient pas de Code

Les publications annuelles et mensuelles de l'IPC contiennent un tableau avec un mapping Code-Label.

Si nous souhaitons analyser l'IPC avec une perspective Divisions/Groupes, nous devons :
- Récupérer le code de chaque élément
- Indiquer son niveau
- Indiquer son parent direct

In [99]:
mapping_file_name = "mapping_code_libelle_panier_IPC.parquet"
mapping_file_path = pathlib.Path(raw_data_path, mapping_file_name)
df_raw_mapping = pd.read_parquet(mapping_file_path)
df_raw_mapping

In [101]:
# Add "code" to identify each element

dc_raw_mapping = {libelle.strip():[] for libelle in df_raw_mapping.libelle.tolist()}
for element in df_raw_mapping.itertuples():
    dc_raw_mapping[element.libelle.strip()].append(element.code)

dc_mapping = {k:min(v, key=len) for k, v in dc_raw_mapping.items()}

pivoted_df["code"] = pivoted_df["produit"].map(dc_mapping)

In [104]:
# Add "parent_code" to identify the parent of each element

def get_parent_code(code):
    if (
        isinstance(code, float) or
        len(code)<=2
    ):
        parent = None
    else:
        parent = code[:-1]
    return parent

pivoted_df["parent_code"] = pivoted_df["code"].apply(get_parent_code)

In [105]:
# Add "type_produit" to identify the level of each element

def get_type_of_element(code):
    if isinstance(code, float):
        type_element = "total"
    elif len(code)<=2:
        type_element = "division"
    else:
        type_element = "groupe"
    return type_element

pivoted_df["type_produit"] = pivoted_df["code"].apply(get_type_of_element)

### Export joined-pivoted data to `.parquet`

In [109]:
clean_file_name = "IPC_base_2017.parquet"
clean_file_path = pathlib.Path(clean_data_path, clean_file_name)
pivoted_df.sort_values(by=["code", "year"]).to_parquet(clean_file_path)

## Data Viz

In [113]:
df = pivoted_df[pivoted_df["type_produit"] == "division"]
px.line(df, x="year", y="value", color="produit")